In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for 
# dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from module4_crud import DataCRUD

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 32839
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = DataCRUD(username, password, host, port, 
                               database, collection)


# class read method must support return of list object and 
# accept projection json input
# sending the read method an empty document requests all 
# documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is 
# going to have an invlaid object type of 'ObjectID' - which will 
# cause the data_table to crash - so we remove it in the dataframe 
# here. The df.drop command allows us to drop the column. If we do 
# not set inplace=True - it will reeturn a new dataframe that 
# does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


imagepath = "Assets/Grazioso Salvare Logo.png" 

image_filename = 'Assets/Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(
        encoded_image.decode()), style={
        'height':'10%', 'width':'10%'}),
    html.Div(),
    html.Hr(),
    html.P("Seungman's Unique Identifier"),
    html.Hr(),

    # Radio buttons to filter data
    dcc.RadioItems(id = 'filter-type',
                   options = ["Water Rescue", "Mountain Rescue", 
                   "Disaster Rescue", "Reset"], 
                   value = "Reset"
                  ),
    html.Hr(),
    # Table to display data
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, 
                                   "deletable": False, 
                                   "selectable": True} 
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         filter_action = "native",
                         row_selectable = "single",
                         column_selectable = "single",
                         selected_rows = [0],
                         selected_columns = [],
                         page_size = 10
                        ),
    html.Br(),
    html.Hr(),
# This sets up the dashboard so that the chart and the 
# geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback (filter options)

@app.callback(
    Output('datatable-id','data'),
    Output('datatable-id','columns'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    
    if filter_type == "Water Rescue":
        filterRes = db.read( {
            "$and": [ { 
                "$or": [
                    {"breed":"Labrador Retriever Mix"},
                    {"breed":"Chesapeake Bay Retriever"},
                    {"breed":"Newfoundland"}
                ]
                },
                {"sex_upon_outcome": "Intact Female"},
                { "$and": [
                    {"age_upon_outcome_in_weeks": {"$gt" : 25}},
                    {"age_upon_outcome_in_weeks": {"$lt" : 157}}    
                ]
                }
            ]
            }
        )
    elif filter_type == "Mountain Rescue":
        filterRes = db.read( { "$and": [
                { "$or": [
                    {"breed":"German Shepherd"},
                    {"breed":"Alaskan Malamute"},
                    {"breed":"Old English Sheepdog"},
                    {"breed":"Siberian Husky"},
                    {"breed":"Rottweiler"},
                ]
                },
                {"sex_upon_outcome": "Intact Male"},
                { "$and": [
                    {"age_upon_outcome_in_weeks": {"$gt" : 25}},
                    {"age_upon_outcome_in_weeks": {"$lt" : 157}}    
                ]
                }
            ]
            }
        )
    elif filter_type == "Disaster Rescue":
        filterRes = db.read( { "$and": [
                { "$or": [
                    {"breed":"Doberman Pinscher"},
                    {"breed":"German Shepherd,"},
                    {"breed":"Golden Retriever"},
                    {"breed":"Bloodhound"},
                    {"breed":"Rottweiler"},
                ]
                },
                {"sex_upon_outcome": "Intact Male"},
                { "$and": [
                    {"age_upon_outcome_in_weeks": {"$gt" : 19}},
                    {"age_upon_outcome_in_weeks": {"$lt" : 301}}    
                ]
                }
            ]
            }
        )
    else:
        filterRes = db.read({})


    df = pd.DataFrame.from_records(filterRes)
    df.drop(columns=['_id'],inplace=True)
    

    columns=[{"name": i, "id": i, "deletable": False, 
              "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    # Pie chart
    df = viewData

    fig = px.pie(df, names = 'breed', 
                           title ='Preferred Animals')
    fig.update_traces(textposition ='inside')
    fig.update_layout(
        uniformtext_minsize = 12, 
        uniformtext_mode ='hide',
        title_x= 0.5,
        title_xanchor = "center", 
        title_yanchor = "top",
    )
    
    return [
       dcc.Graph(            
           figure = fig
       )    
    ]
    
#This callback will highlight a cell on the data table when the 
# user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the 
# selected data entry
# derived_virtual_data will be the set of data available from the 
# datatable in the form of a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the 
# table in the form of a list. For this application, we are only 
# permitting single row selection so there is only one value in 
# the list.
# The iloc method allows for a row, column notation to pull data 
# from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be 
    # converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the 
            # map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], 
                      children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
                      ]
                     )
               ])
    ]



app.run_server(debug=True)


Collection initialized: animals
Dash app running on http://127.0.0.1:8290/
